<a href="https://colab.research.google.com/github/yhp2205/SQL/blob/main/ch_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 8강 여러 개의 테이블 조작하기

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### 여러 개의 테이블을 세로로 결합

세로로 결합하기 위한 데이터 테이블을 app1_mst_users와 app2_mst_users로 나누어 작성하겠습니다. 

In [3]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.app1_mst_users;
CREATE TABLE sqldata.app1_mst_users (
    user_id string
  , name    string
  , email   string
);

INSERT INTO sqldata.app1_mst_users
VALUES
    ('U001', 'Sato'  , 'sato@example.com'  )
  , ('U002', 'Suzuki', 'suzuki@example.com')
;

DROP TABLE IF EXISTS sqldata.app2_mst_users;
CREATE TABLE sqldata.app2_mst_users (
    user_id string
  , name    string
  , phone   string
);

INSERT INTO sqldata.app2_mst_users
VALUES
    ('U001', 'Ito'   , '080-xxxx-xxxx')
  , ('U002', 'Tanaka', '070-xxxx-xxxx')
;


""


두 데이터의 구조가 user_id, name은 동일하고 phone과 email 등의 컬럼이 다른 비슷한 구조를 지니고 있기 때문에 UNION ALL 구문을 사용하여 여러 개의 테이블을 세로로 결합할 수 있습니다. 한쪽 테이블에만 존재하는 컬럼은 SELECT 구문에서 제외하거나, 디폴트 값을 줌으로써 처리하면 됩니다.

In [4]:
%%bigquery --project mygcpproject-340112
  -- 8.1 UNION ALL 구문을 사용해 테이블을 세로로 결합하는 쿼리
  SELECT 'app1' AS app_name, user_id, name, email FROM sqldata.app1_mst_users
UNION ALL
  SELECT 'app2' AS app_name, user_id, name, NULL AS email FROM sqldata.app2_mst_users;

,app_name,user_id,name,email
0,app1,U001,Sato,sato@example.com
1,app1,U002,Suzuki,suzuki@example.com
2,app2,U001,Ito,None
3,app2,U002,Tanaka,None


다음과 같이 phone 컬럼을 제외하고, email 컬럼에 디폴트 값으로 NULL 값을 지정함으로써 두 테이블을 세로로 결합해보았습니다. 

### 여러 개의 테이블을 가로로 정렬

In [5]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.mst_categories;
CREATE TABLE sqldata.mst_categories (
    category_id integer
  , name        string
);

INSERT INTO sqldata.mst_categories
VALUES
    (1, 'dvd' )
  , (2, 'cd'  )
  , (3, 'book')
;

DROP TABLE IF EXISTS sqldata.category_sales;
CREATE TABLE sqldata.category_sales (
    category_id integer
  , sales       integer
);

INSERT INTO sqldata.category_sales
VALUES
    (1, 850000)
  , (2, 500000)
;

DROP TABLE IF EXISTS sqldata.product_sale_ranking;
CREATE TABLE sqldata.product_sale_ranking (
    category_id integer
  , rank        integer
  , product_id  string
  , sales       integer
);

INSERT INTO sqldata.product_sale_ranking
VALUES
    (1, 1, 'D001', 50000)
  , (1, 2, 'D002', 20000)
  , (1, 3, 'D003', 10000)
  , (2, 1, 'C001', 30000)
  , (2, 2, 'C002', 20000)
  , (2, 3, 'C003', 10000)
;


""


여러 개의 테이블을 가로로 정렬할 때에는 JOIN을 사용하면 편리합니다.

In [6]:
%%bigquery --project mygcpproject-340112
  -- 8.2 여러 개의 테이블을 결합해서 가로로 정렬하는 쿼리
SELECT
    m.category_id
    , m.name
    , s.sales
    , r.product_id AS sale_product
FROM
    sqldata.mst_categories AS m
  JOIN
    -- 카테고리별로 매출액 결합하기
    sqldata.category_sales AS s
    ON m.category_id = s.category_id
  JOIN
    -- 카테고리별로 상품 결합하기
    sqldata.product_sale_ranking AS r
    ON m.category_id = r.category_id
;

,category_id,name,sales,sale_product
0,2,cd,500000,C002
1,2,cd,500000,C003
2,2,cd,500000,C001
3,1,dvd,850000,D001
4,1,dvd,850000,D002
5,1,dvd,850000,D003


단순히 category_id로 테이블을 결합한 결과, book 카테고리가 결합되지 못하고 여러 상품 데이터가 소실된 것을 볼 수 있습니다.  
마스터 테이블의 행 수를 변경하지 않고 데이터를 가로 정렬하기 위해서는 LEFT JOIN을 사용하여 결합하지 못한 레코드를 유지하면서, 결합할 레코드가 1개 이하가 되도록 조건을 지정해야 합니다. 

In [7]:
%%bigquery --project mygcpproject-340112
  -- 8.3 마스터 테이블의 행 수를 변경하지 않고 여러 개의 테이블을 가로로 정렬하는 쿼리
SELECT
  m.category_id
  , m.name
  , s.sales
  , r.product_id AS top_sale_product
FROM
    sqldata.mst_categories AS m
  -- LEFT JOIN 을 사용해서 결합한 레코드를 남김
  LEFT JOIN
    -- 카테고리별 매출액 결합하기
    sqldata.category_sales AS s
    ON m.category_id = s.category_id
  -- LEFT JOIN을 사용해서 결합하지 못한 레코드를 남김
  LEFT JOIN
    -- 카테고리별 최고 매출 상품 하나만 추출해서 결합하기
    sqldata.product_sale_ranking AS r
    ON m.category_id = r.category_id
    AND r.rank = 1
;

,category_id,name,sales,top_sale_product
0,2,cd,500000.0,C001
1,1,dvd,850000.0,D001
2,3,book,NaN,None


앞서 코드에서 rank = 1 을 지정하여 매출 순위의 1위 상품이라는 조건을 걸어 카테고리가 여러 행이 되지 않도록 하였습니다.  
  

In [9]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.mst_users_with_card_number;
CREATE TABLE sqldata.mst_users_with_card_number (
    user_id     string
  , card_number string
);

INSERT INTO sqldata.mst_users_with_card_number
VALUES
    ('U001', '1234-xxxx-xxxx-xxxx')
  , ('U002', NULL                 )
  , ('U003', '5678-xxxx-xxxx-xxxx')
;

DROP TABLE IF EXISTS sqldata.purchase_log;
CREATE TABLE sqldata.purchase_log (
    purchase_id integer
  , user_id     string
  , amount      integer
  , stamp       string
);

INSERT INTO sqldata.purchase_log
VALUES
    (10001, 'U001', 200, '2017-01-30 10:00:00')
  , (10002, 'U001', 500, '2017-02-10 10:00:00')
  , (10003, 'U001', 200, '2017-02-12 10:00:00')
  , (10004, 'U002', 800, '2017-03-01 10:00:00')
  , (10005, 'U002', 400, '2017-03-02 10:00:00')
;


""


In [10]:
%%bigquery --project mygcpproject-340112
  -- 8.5 신용 카드 등록과 구매 이력 유무를 0과 1이라는 플래그로 나타내는 쿼리
SELECT
  m.user_id
  , m.card_number
  , COUNT(p.user_id) AS purchase_count
    -- 신용 카드 번호를 등록한 경우 1, 등록하지 않은 경우 0으로 표현하기
  , CASE WHEN m.card_number IS NOT NULL THEN 1 ELSE 0 END AS has_card
    -- 구매 이력이 있는 경우 1, 없는 경우 0으로 표현하기
  , SIGN(COUNT(p.user_id)) AS has_purchased
FROM
    sqldata.mst_users_with_card_number AS m
  LEFT JOIN
    sqldata.purchase_log AS p
    ON m.user_id = p.user_id
GROUP BY m.user_id, m.card_number
;

,user_id,card_number,purchase_count,has_card,has_purchased
0,U003,5678-xxxx-xxxx-xxxx,0,1,0
1,U001,1234-xxxx-xxxx-xxxx,3,1,1
2,U002,None,2,0,1


CASE 식과 SIGN 함수로 신용카드 등록과 구매 이력 유부를 0또는 1로 나타내는 쿼리입니다. 

In [11]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.product_sales;
CREATE TABLE sqldata.product_sales (
    category_name string
  , product_id    string
  , sales         integer
);

INSERT INTO sqldata.product_sales
VALUES
    ('dvd' , 'D001', 50000)
  , ('dvd' , 'D002', 20000)
  , ('dvd' , 'D003', 10000)
  , ('cd'  , 'C001', 30000)
  , ('cd'  , 'C002', 20000)
  , ('cd'  , 'C003', 10000)
  , ('book', 'B001', 20000)
  , ('book', 'B002', 15000)
  , ('book', 'B003', 10000)
  , ('book', 'B004',  5000)
;


""


복잡한 처리를 하는 SQL 문을 작성할 때 서브 쿼리의 중첩이 많아지기 때문에 쿼리의 가독성을 높이기 위해 공통 테이블 식을 사용하기도 합니다. 일시적으로 테이블에 이름을 붙여 재사용할 수 있습니다. 앞선 테이블인 product_sales를 사용하여 카테고리별 순위를 가로로 전개하고 카테고리 이름 별 상품 매출 순위를 한 번에 볼 수 있는 형식으로 변환하겠습니다. 

In [12]:
%%bigquery --project mygcpproject-340112
  -- 8.6 카테고리별 순위를 추가한 테이블에 이름 붙이기
WITH
product_sale_ranking AS (
  SELECT
      category_name
    , product_id
    , sales
    , ROW_NUMBER() OVER(PARTITION BY category_name ORDER BY sales DESC) AS rank
  FROM
    sqldata.product_sales
)
SELECT *
FROM sqldata.product_sale_ranking
;

,category_id,rank,product_id,sales
0,1,1,D001,50000
1,1,2,D002,20000
2,2,2,C002,20000
3,1,3,D003,10000
4,2,3,C003,10000
5,2,1,C001,30000


ROW_NUMBER 로 카테고리별 순위를 붙이고 CTE구문('WITH <테이블이름> AS (SELECT ~)' 형태의 구문)을 사용하여 만든 테이블에 product_sale_ranking 이라는 이름을 붙입니다.  
이 테이블을 결합하여 카테고리 수 만큼 펼칩니다. 카테고리들에 포함된 상품의 수가 다르므로 최대 상품 수에 맞는 결과를 계산할 수 있게 유니크한 목록을 계산해보겠습니다.  
WITH 구문을 사용하여 여러 테이블을 정의해야하기 때문에 쉼표를 사용합니다.

In [15]:
%%bigquery --project mygcpproject-340112
  -- 8.7 카테고리들의 순위에서 유니크한 순위 목록을 계산하는 쿼리
WITH
product_sale_ranking AS (
  SELECT
      category_name
    , product_id
    , sales
    , ROW_NUMBER() OVER(PARTITION BY category_name ORDER BY sales DESC) AS rank
  FROM
    sqldata.product_sales
)
, mst_rank AS (
  SELECT DISTINCT rank
  FROM sqldata.product_sale_ranking
)
SELECT *
FROM mst_rank
;

,rank
0,1
1,2
2,3


유니크한 순위 목록을 구했으면 이어서 카테고리 순위를 LEFT JOIN 으로 결합합니다. 

In [18]:
%%bigquery --project mygcpproject-340112
  -- 8.8 카테고리들의 순위를 횡단적으로 출력하는 쿼리
WITH
product_sale_ranking AS (
  SELECT
      category_name
    , product_id
    , sales
    , ROW_NUMBER() OVER(PARTITION BY category_name ORDER BY sales DESC) AS rank
  FROM
    sqldata.product_sales
)
, mst_rank AS (
  SELECT DISTINCT rank
  FROM sqldata.product_sale_ranking
)
SELECT
    m.rank
  , r1.product_id AS dvd
  , r1.sales      AS dvd_sales
  , r2.product_id AS cd
  , r2.sales      AS cd_sales
  , r3.product_id AS book
  , r3.sales      AS book_sales
FROM
    mst_rank AS m
  LEFT JOIN
    product_sale_ranking AS r1
    ON m.rank = r1.rank
    AND r1.category_name = 'dvd'
  LEFT JOIN
    product_sale_ranking AS r2
    ON m.rank = r2.rank
    AND r2.category_name = 'cd'
  LEFT JOIN
    product_sale_ranking AS r3
    ON m.rank = r3.rank
    AND r3.category_name = 'book'
ORDER BY m.rank
;

,rank,dvd,dvd_sales,cd,cd_sales,book,book_sales
0,1,D001,50000,C001,30000,B001,20000
1,2,D002,20000,C002,20000,B002,15000
2,3,D003,10000,C003,10000,B003,10000


코드 값과 레이블 조합을 유사 테이블로 만들어 집계하는 방법을 알아보겠습니다.  
SELECT 구문을 사용하여 유사 테이블을 만들어보겠습니다. 

In [25]:
%%bigquery --project mygcpproject-340112
  -- 8.9 디바이스 ID와 이름의 마스터 테이블을 만드는 쿼리
WITH
mst_devices AS (
            SELECT 1 AS device_id, 'PC' AS device_name
  UNION ALL SELECT 2 AS device_id, 'SP' AS device_name
  UNION ALL SELECT 3 AS device_id, '애플리케이션' AS device_name
)
SELECT *
FROM mst_devices
;

,device_id,device_name
0,1,PC
1,2,SP
2,3,애플리케이션


앞의 의사 테이블을 사용하여 다시 작성해보겠습니다.

In [27]:
%%bigquery --project mygcpproject-340112
  -- 8.10 의사 테이블을 사용해 코드를 레이블로 변환하는 쿼리
WITH
mst_devices AS (
            SELECT 1 AS device_id, 'PC' AS device_name
  UNION ALL SELECT 2 AS device_id, 'SP' AS device_name
  UNION ALL SELECT 3 AS device_id, '애플리케이션' AS device_name
)
SELECT
    u.user_id
  , d.device_name
FROM
    sqldata.mst_users AS u
  LEFT JOIN
    mst_devices AS d
    ON u.register_device = d.device_id
;

,user_id,device_name
0,U001,PC
1,U002,SP
2,U003,애플리케이션
